# Sentence Mining in a Serbian Web Corpus

Demonstration of how pyspark can load streaming xml data, lightly parse, and filter results. 

We produce examples of the possessive prounouns of Serbian for several cases, allowing students to study all forms of their declension.

In [1]:
import pyspark
from pyspark.sql.types import StringType
import time
from itertools import islice
from tqdm.notebook import tqdm
import numpy as np
import codecs
import glob

The first 100,000 lines of the corpus are included in this repo (License: CC BY-SA 4.0)

Download additional data from https://www.clarin.si/repository/xmlui/handle/11356/1063

If you do, then to speed things up, you can turn off processing of the entire multigigabyte corpus and instead just look at the first 100,000 lines with the following flag:

In [2]:
PROCESS_ENTIRE_CORPUS = True

In [3]:
corpus_files = glob.glob('data/srWaC*.xml'); print(corpus_files)

# Settings for processing smaller chunk of data
num_of_th = 48; repartition_size = num_of_th*4
chunk_size = 100000

MAX_WORDS_IN_SENTENCE = 6

['data/srWaC1.1.01.xml']


Start the pyspark cluster (locally)

In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext


sc = SparkContext(master = "local[20]").getOrCreate()
spark = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 14:14:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Create the rdd

In [5]:
%%time
if PROCESS_ENTIRE_CORPUS:
    small_chunk = False
    file_rdd = spark.read.text(corpus_files, wholetext=False)
else:
    small_chunk = True
    file_rdd = spark.read.text(corpus_files[0], wholetext=False)
    file_chunk = file_rdd.take(chunk_size) 
    file_rdd = sc.parallelize(file_chunk) 

CPU times: user 3.04 ms, sys: 2.82 ms, total: 5.86 ms
Wall time: 2.49 s


In [6]:
# Parses the xml stream to accumulate lines corresponding to the same sentence
def quickparse(part, max_len=None):
    
    state = None
    accumulator = []
    readlines = 0
    
    for line in part:
        line = line['value']
        
        readlines +=1
        
        if line == '<s>':
            state = '<s>'
            accumulator = []
            continue
        if line == '</s>':
            state = '</s>'
            if len(accumulator) < 3: continue #  or len(accumulator) > 6: continue
            tr = ['|'.join(_) for _ in np.array(accumulator).T]
            yield tr
        if state == "<s>" and line != '<g/>':
            accumulator.append(line.split('\t'))
            if max_len is not None and len(accumulator) >= max_len: state = None

Create a dataframe with columns for the different word forms: text, reformat, lemma, and language properties. Display the first 10 which have at least one possessive pronoun in it.

In [7]:
if small_chunk:
    df = file_rdd.mapPartitions(lambda x: quickparse(x, max_len=MAX_WORDS_IN_SENTENCE)).toDF(['text','reformat', 'lemma', 'feats'])
else:
    df = file_rdd.rdd.mapPartitions(lambda x: quickparse(x, max_len=MAX_WORDS_IN_SENTENCE)).toDF(['text','reformat', 'lemma', 'feats'])


df1 = df.filter(df['feats'].rlike(r'Ps\w\w\w[n|a]')).cache()


df1.show(10)

+--------------------+--------------------+--------------------+--------------------+
|                text|            reformat|               lemma|               feats|
+--------------------+--------------------+--------------------+--------------------+
|Za|njegovu|smrt|s...|za|njegovu|smrt|s...|za|njegov|smrt|bi...|Sa|Ps3fsa|Ncfsa|V...|
|Moj|stav|je|da|mi|je|moj|stav|je|da|mi|je|moj|stav|biti|da|...|Ps1msn|Ncmsn|Var3...|
|Naša|vera|je|čist...|naša|vera|je|čist...|naš|vera|biti|čis...|Ps1fsn|Ncfsn|Var3...|
|Tvoje|preduzeće|p...|tvoje|preduzeće|p...|tvoj|preduzeće|pr...|Ps2nsn|Ncnsn|Vmr3...|
|Njihov|repistolar...|njihov|repistolar...|njihov|repistolar...|Ps3msn|Agpmsayn|N...|
|Međutim|,|njegov|...|međutim|,|njegov|...|međutim|,|njegov|...|Rgp|Z|Ps3msn|Ncms...|
|Moja|beba|krivi|g...|moja|beba|krivi|g...|moj|beba|kriviti|...|Ps1fsn|Ncfsn|Vmr3...|
|"|Provokacija|"|j...|"|provokacija|"|j...|"|provokacija|"|b...|Z|Ncfsn|Z|Var3s|P...|
|Za|učenike|i|njih...|za|učenike|i|njih...|za|učenik|i

## Sentence Generation

We want to collect many sentences for language practice. Let's enumerate the different cases and genders we want to consider. We will also filter the sentences for known bad words.

In [8]:
bad_words_rot13 = ['cravf', 'xhenp', 'wrongv', 'cvčxn', 'xyvgbevf', 'oenqnivpn', 'qbwxr', 'oybiwbo', 'fabšnw', 'bqwrovgv', 'hzhxavgv']
bad_words = [codecs.decode(_, 'rot13') for _ in bad_words_rot13]

In [9]:
property_filters = [
    {'lemma':'njen', 'Case':'Acc', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'njen', 'Case':'Acc', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'njen', 'Case':'Acc', 'Gender':'Fem', 'Number':'Sing'},
    {'lemma':'njen', 'Case':'Nom', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'njen', 'Case':'Nom', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'njen', 'Case':'Nom', 'Gender':'Fem', 'Number':'Sing'},
    
    {'lemma':'moj', 'Case':'Acc', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'moj', 'Case':'Acc', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'moj', 'Case':'Acc', 'Gender':'Fem', 'Number':'Sing'},
    {'lemma':'moj', 'Case':'Nom', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'moj', 'Case':'Nom', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'moj', 'Case':'Nom', 'Gender':'Fem', 'Number':'Sing'},
    
    {'lemma':'tvoj', 'Case':'Acc', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'tvoj', 'Case':'Acc', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'tvoj', 'Case':'Acc', 'Gender':'Fem', 'Number':'Sing'},
    {'lemma':'tvoj', 'Case':'Nom', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'tvoj', 'Case':'Nom', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'tvoj', 'Case':'Nom', 'Gender':'Fem', 'Number':'Sing'},
    
    {'lemma':'njegov', 'Case':'Acc', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'njegov', 'Case':'Acc', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'njegov', 'Case':'Acc', 'Gender':'Fem', 'Number':'Sing'},
    {'lemma':'njegov', 'Case':'Nom', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'njegov', 'Case':'Nom', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'njegov', 'Case':'Nom', 'Gender':'Fem', 'Number':'Sing'},
    
    {'lemma':'naš', 'Case':'Acc', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'naš', 'Case':'Acc', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'naš', 'Case':'Acc', 'Gender':'Fem', 'Number':'Sing'},
    {'lemma':'naš', 'Case':'Nom', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'naš', 'Case':'Nom', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'naš', 'Case':'Nom', 'Gender':'Fem', 'Number':'Sing'},

    {'lemma':'vaš', 'Case':'Acc', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'vaš', 'Case':'Acc', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'vaš', 'Case':'Acc', 'Gender':'Fem', 'Number':'Sing'},
    {'lemma':'vaš', 'Case':'Nom', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'vaš', 'Case':'Nom', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'vaš', 'Case':'Nom', 'Gender':'Fem', 'Number':'Sing'},
    
    {'lemma':'njihov', 'Case':'Acc', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'njihov', 'Case':'Acc', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'njihov', 'Case':'Acc', 'Gender':'Fem', 'Number':'Sing'},
    {'lemma':'njihov', 'Case':'Nom', 'Gender':'Masc', 'Number':'Sing'},
    {'lemma':'njihov', 'Case':'Nom', 'Gender':'Neut', 'Number':'Sing'},
    {'lemma':'njihov', 'Case':'Nom', 'Gender':'Fem', 'Number':'Sing'},
]

In [10]:
# pretty printing of the metadata for a word record
def pprint(x):
    text = x['text'].split('|')
    feat = x['feats'].split('|')
    output = text[0]
    for t,f in zip(text[1:],feat[1:]):
        if f != 'Z': 
            output += ' ' + t
        else:
            output += t
            
    return output

In [11]:
%%time 

SENTENCES_TO_MINE = 20 # return at most this number of sentences

pbar = tqdm(property_filters)
with open('final_results.txt','w') as fout:
    for pf in pbar:
        print(pf)
        fout.write(str(pf)+'\n')
        lemma = pf['lemma']
        case = pf['Case'][0].lower()
        gender = pf['Gender'][0].lower()
        number = pf['Number'][0].lower()
        ffeat = r'Ps' + r'\w' + gender + number + case 

        df2 = df1.filter(df1['feats'].rlike(ffeat))
        df3 = df2.filter(df2['lemma'].rlike(lemma))

        taken =  df3.take(SENTENCES_TO_MINE)
        for _ in list(set(taken)):
            disallowed = np.any([b in _['lemma'] for b in bad_words])
            if not disallowed:
                fout.write('\t'+pprint(_)+'\n')
                print('\t',pprint(_))
            else: 
                continue
                print('bad words found!', pprint(_))    

  0%|          | 0/42 [00:00<?, ?it/s]

{'lemma': 'njen', 'Case': 'Acc', 'Gender': 'Masc', 'Number': 'Sing'}
{'lemma': 'njen', 'Case': 'Acc', 'Gender': 'Neut', 'Number': 'Sing'}
	 Njeno mišljenje deli i Ivan Mikalački
{'lemma': 'njen', 'Case': 'Acc', 'Gender': 'Fem', 'Number': 'Sing'}
{'lemma': 'njen', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing'}
{'lemma': 'njen', 'Case': 'Nom', 'Gender': 'Neut', 'Number': 'Sing'}
	 " Provokacija" je njeno drugo
{'lemma': 'njen', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}
	 Sonja Biserko i njena NVO misle
	 Njena kotlina je intezivnije naseljena tokom
{'lemma': 'moj', 'Case': 'Acc', 'Gender': 'Masc', 'Number': 'Sing'}
{'lemma': 'moj', 'Case': 'Acc', 'Gender': 'Neut', 'Number': 'Sing'}
{'lemma': 'moj', 'Case': 'Acc', 'Gender': 'Fem', 'Number': 'Sing'}
{'lemma': 'moj', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing'}
	 Sumnjam, moj um je i
	 moj utisak je da je ovo
	 Moj stav je da mi je
	 Znaci moj licni zakljucak je da
	 Takve okolnosti učvršćuju moj stav da
	 Moj prijatelj uv

Not every case/gender/number is present in the first 100,000 lines.